# Lecture 3 Overview

Framing wrap-up
- [Sentinel-based framing](#sentinel-based-framing)
- Clock-based framing

Error handling through redundancy

Hamming Distance
- When we can DETECT
- When we can CORRECT

# Project 1 is out

Implement simple link layer protocol (Layer 2)

**Framing and handling dropped frames**

Rationale?
- Help understand how NIC work
- Link layers change/update frequently. You may be working on one!

# What's in a Frame?

`[-Header---|-----Payload----------|-Trailer-]`

Wraps payload bits up with some additional info
- Header: addressing info
- Trailer: checksum or CRC (more in next lecture)

Basic Unit of Communication
- All or nothing (Link delivers either entire frame payload or none)
- **Maximum Transmission Unit** (MTU)

Some link layers require absense of frames
- i.e. **minimum gap** between frames

# Identifying Frames

First Task: **delineate** frames
- Starting/Ending is impo to know
- Otherwise, errors from misinterpretation of data stream

Methods
1. [Fixed length (bits) frames](#fixed-length-frames)

- Explicitly delimited frames:
  - 2. [Length-based framing](#length-based-framing)
  - 3. [Sentinel-based framing](#sentinel-based-framing)

4. Fixed duration (seconds) frames

## Fixed-Length Frames

Easy to manage for receiver
- Well understood buffering reqs

Introduces inefficiencies for var length payloads
- Larger paylods --> **fragmentated** across many frames
- May waste bandwidth (for every X bits, you have to send N header bits)
- Very COMMON implementation inside switches

Requires explicit design tradeoff
- ATM uses 53B frames (cells)
  - 48 + 5

> Asynchronous Transfer Mode (ATM) Phone network, not ATM machine for money

> Story time: geopolitical argument over length (france 32, us 64, etc.) resolved by picking the middle (48) haha. No one was happy -- this was in the 80s/90s

## Length-Based Framing

`[--Start--|-Length-|-----Payload--------]`

To avoid overhead, we'd like var length frames
- Each frame declares how long it is
- Examples:
  - DECNet
  - DDCMP

> N bit length field ==> (header+N+$2^N$) MTU 

What's the isisue w/ expliciti length field?
- Must correctly READ that field (bad if corrupted)
  - need to decode WHILE receiving
- Still need to identify the beginning...

> Issues: underreading/overreading the frame. Will either misinterpret middle of frame as start of next. Or misinterpret next frame as a continuation of this frame

## Sentinel-based Framing

> Allows for variable length frames by it's own nature

Idea: mark start/end of frame w/ special "market"
- Byte pattern, bit pattern, signal pattern

> Means reserved markers CANNOT APPEAR IN DATA

Two solution
- Special non-data symbol used to convey sentinel
  - Impact on efficiency (can't use symbol for data) of code
- **Stuffing**
  - Dynamically remove marker bit patterns from data stream
  - Receiver "unstuffs" datastream to reconstruct original data



### Stuffing

Insert bytes/bits into datastream to make sure that sentinel (flag) does NOT appear in payload

![](images/2023-01-18-10-20-49.png)

#### Bit-level Stuffing

Avoid sentinel flag bit pattern in payload data
- commonly, sentinel flag is a bit pattern `0x7E` (`0111110`)
- Invented for SDLC/HDLC (IBM layer 2), now standard pattern

Sender: any time FIVE ones appear in outgoing data, insert a zero (results in `0111110`)

![stuffed-bits diagram](images/2023-01-18-10-25-29.png)

Receiver: any time FIVE ones appear, remove next zero
- If there is no zero, there will either be six 1s and a 0 (sentinel)
- It declares an error condition!
- Note bit pattern that cannot appear is `0x7F` (`01111111`)

Worst case efficiency is when you send a lot of `011111` --> you end up transmitting an extra bit for every Byte

#### Byte Stuffing

Upping sentinel to Byte (character) level
- Generally have 2 diff flags, **STX** and **ETX**
- Found in:
  - PPP
  - DDCMP
  - BISYNC
  - Embedded Systems
  - etc.

Need to stuff if either appears in payload
- prefix byte with special character, **DLE** (data-link escape)
- new problem: what if DLE appears in payload?

Stuff DLE w/ DLE!
- worst case is 50% efficient (sending all DLEs)

![escape byte diagram](./images/2023-01-18-10-30-11.png)

### Clock signal-based Framing

So far, we've based framing on what's on the wire
- Any bit errors --> throw off framing (dropped bits)
- What happens with missed delimiter? Spurious delimiter?

Alterntaive is to base framing on external clock signal
- Use some signal to indicate beginning of frame, and wait fixed time until frame ends
- SONET (metro optical network) does this, among others

Significant engineering tradeoffs
- (+) No extra bits in datastream iself
- (-) Need a tight clock synch b/w sender/receiver

> Optimized for video streaming, not so much for stuff like SSH (not fast/consistently sending bits)

##### SONET

Synchronous Optical NETwork
- historically used to backhaul tele + internet traffic
- Engineering goal: reduce delay & buffering

All frames take same amt of time
- Independent of bitrate
- Can have atomic clock at each end of link ==> guarantee sync

Each frame still starts with special sentinel bits
- Helps sync clocks --> look for periodic signal bits
- No need to stuff; signal pattern is unlikely

> datastream isn't likely to be periodic

#### Things can still go wrong

Clock drift --> frame boudnaries get fuzzy
- may not have atomic clocks --> can desync
- Result: read end of one frame and beginning of next :(

What happens if there are bit errors on channel?
- Misinterpret clock sync signals (sentinels) as data or vice versa

In general, need some way to make sure we're OK
- Error detection, and perhaps correction

> Cosmic rays from sun --> EM interference on link that causes bit flips

# Error Detection

Implemented at many layers (end-to-end arg) BUT mainly focused on link-layer techniques today

![Error Detection img](images/2023-01-18-10-46-10.png)

## Basic Idea

Problem: header/payload is not self-verifying
- Every bistring is potentially legitimate
- Hence, any errors/changes in set of bits = equally legit

Solution: reduce set of potential bistrings
- not every bistring is allowable
- receiving disallowed bistring triggers warning => original its garbled in transit

Key question: which bitstrings are allowed?

## Codewords

Starting simple: fixed-length bitstring
- reduce to n-bit substrings